In [7]:
import sys
import time
import heapq
import math
import gc
import numpy as np
import pandas as pd
from collections import defaultdict
import surprise as sp

In [24]:
#trainset
filePath = "/data/fjsdata/BNMF/ml-1m.train.rating" 
data = pd.read_csv(filePath, sep='\t', header=None, names=['user', 'item', 'rating'], \
                                 usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.float})
#data['rating']=data['rating'].apply(lambda x: 1.0 if float(x)>0.0 else 0.0)
maxu, maxi = data['user'].max()+1, data['item'].max()+1
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
                  (data.shape[0], maxu, maxi, data.shape[0]/(maxu*maxi)))
trainset = data.values.tolist()
trainMat = np.zeros([maxu, maxi], dtype=np.float32)
for u,i,r in trainset:
    trainMat[int(u)][int(i)] = float(r)
#testset
testset = []
filePath = "/data/fjsdata/BNMF/ml-1m.test.negative" 
with open(filePath, 'r') as fd:
    line = fd.readline()
    while line != None and line != '':
        arr = line.split('\t')
        u = eval(arr[0])[0]
        testset.append([u, eval(arr[0])[1], 1.0])#first is one postive item
        for i in arr[1:]:
            testset.append([u, int(i), 0.0]) #99 negative items
        line = fd.readline()
print ('The length of Testset: %d'%(len(testset)))

Dataset Statistics: Interaction = 994169, User = 6040, Item = 3706, Sparsity = 0.0444
The length of Testset: 604000


In [25]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [27]:
reader = sp.Reader(rating_scale=(0, 5))
spdata = sp.Dataset.load_from_df(data,reader)
trainset_svd = spdata.build_full_trainset()
for K in [5,10,15,20]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#
    algo.fit(trainset_svd)
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (K, hitratio, K, ndcg))

HR@5=0.259272, NDCG@5=0.127297
HR@10=0.258609, NDCG@10=0.125978
HR@15=0.259106, NDCG@15=0.126195
HR@20=0.258775, NDCG@20=0.126813


In [31]:
class BayesianMatrixFactorization():
    """
    Bayesian Matrix Factorization model
    R = PxQ
    p ~ N(p|0, alpha^(-1)I)
    q ~ N(q|0, alpha^(-1)I)
    r = p @ q
    t ~ N(r|p @ q, beta^(-1))
    """

    def __init__(self, alpha_p:float=1., alpha_q:float=1., beta:float=1.):
        """
        ----------
        n_u, n_i: the number of users and items, respectively.
        k : the number of latent factors
        """
        self.alpha_p = alpha_p
        self.alpha_q = alpha_q
        self.beta = beta
        #posterior of p,q 
        self.pos_mean_p = None
        self.pos_precision_p = None
        self.pos_mean_q = None
        self.pos_precision_q = None

    def fit(self, R:np.ndarray, k:int=5):
        """
        bayesian update of parameters given training dataset
        Parameters
        ----------
        R : (u,i) np.ndarray
            training data independent variable, u is the number of users, i is the number of items.
        k : int, the number of latent factors.
        """
        #1. generate matrices P, Q
        P = np.random.normal(0,self.alpha_p,(R.shape[0],k))#uxk
        Q = np.random.normal(0,self.alpha_q,(R.shape[1],k))#ixk
        #2.calculate the posterior with analytical solution
        self.pos_precision_p = self.alpha_p + self.beta * Q @ Q.T # ixi
        self.pos_mean_p = self.beta * R @ np.linalg.inv(self.pos_precision_p) @ Q # uxi,ixi,ixk -> uxk
        self.pos_precision_q = self.alpha_q + self.beta * P @ P.T # uxu
        self.pos_mean_q = self.beta * R.T @ np.linalg.inv(self.pos_precision_q) @ P # ixu,uxu,uxk -> ixk

    def predict(self, sample_size:int=None):
        """
        return mean  of predictive distribution
        Parameters
        ----------
        sample_size : int, optional
            number of samples to draw from the predictive distribution
            (the default is None, no sampling from the distribution)
        Returns
        -------
        R_pred : (u,i) np.ndarray
            mean of the predictive distribution
        R_pred_sample : (u,i,sample_size) np.ndarray
            samples from the predictive distribution
        """
        if sample_size is not None:
            p_sample = np.random.multivariate_normal(self.pos_mean_p, np.linalg.inv(self.pos_precision_q), size=sample_size)
            q_sample = np.random.multivariate_normal(self.pos_mean_q, np.linalg.inv(self.pos_precision_p), size=sample_size)
            R_pred_sample_list=[]
            for i in range(sample_size): 
                R_pred_sample_list.append( np.dot(p_sample, q_sample.T) )
            R_pred_sample = np.mean(np.array(R_pred_sample_list), axis=0)
            return  R_pred_sample #uxi
        
        R_pred = self.pos_mean_p @ self.pos_mean_q.T #R = PxQ
        return R_pred #uxi


for K in [5,10,15,20]:#iterations epoches
    bmf = BayesianMatrixFactorization()
    bmf.fit(R=trainMat, k=K)
    R_pred = bmf.predict()
    hits = []
    ndcgs = []
    for c in range(0,maxu):#6040
        scorelist = []
        gtItem = -1
        for u,i,r in testset[c*100:(c+1)*100]:#604000
            if r == 1.0: gtItem = i
            est = R_pred[int(u)][int(i)]
            scorelist.append([i,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(30, map_item_score, key=map_item_score.get)#topn=20
        hr = getHitRatio(ranklist, gtItem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, gtItem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("HR@%d=%.6f, NDCG@%d=%.6f" % (K, hitratio, K, ndcg))

HR@5=0.424338, NDCG@5=0.166753
HR@10=0.409934, NDCG@10=0.160615
HR@15=0.424172, NDCG@15=0.173028
HR@20=0.413576, NDCG@20=0.162718
